# Readme
### This notebook is just for doodling, the correct implementation for all of the functionality in this notebook, then check src/main_experiment.py.run()

In [35]:
import dill
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import defaultdict

In [5]:
main_flder = "D:\\PhD\\codes\\behavioralProject\\models\\20210515_201141"

In [7]:
fs_file = os.path.join(main_flder, 'FS_obj.p')
ml_file = os.path.join(main_flder, 'ML_obj.p')
df_file = os.path.join(main_flder, 'group_df_afterFixation.csv')

# Load saved objects 

In [9]:
with open(fs_file, 'rb') as f:
    fs_obj = dill.load(f)
    
with open(ml_file, 'rb') as f:
    ml_obj = dill.load(f)
    
df = pd.read_csv(df_file, index_col='subj_id')

# Create the selected feature matrix out of the dataframe

In [12]:
dfs.columns

Index(['area_lbankssts_medM2080', 'area_lbankssts_medP2080',
       'area_lcaudalanteriorcingulate_medM2080',
       'area_lcaudalanteriorcingulate_medP2080',
       'area_lcaudalmiddlefrontal_medM2080',
       'area_lcaudalmiddlefrontal_medP2080', 'area_lcuneus_medM2080',
       'area_lcuneus_medP2080', 'area_lentorhinal_medM2080',
       'area_lentorhinal_medP2080',
       ...
       'volume_rsuperiortemporal_medP2080', 'volume_rsupramarginal_medM2080',
       'volume_rsupramarginal_medP2080', 'volume_rtemporalpole_medM2080',
       'volume_rtemporalpole_medP2080', 'volume_rtransversetemporal_medM2080',
       'volume_rtransversetemporal_medP2080', 'SRS_COMMUNICATION_T',
       'categories_COMMUNICATION', 'my_labels'],
      dtype='object', length=547)

In [13]:
dfs = df.drop(['SEX','AGE_AT_SCAN ', 'DX_GROUP','categories_COMMUNICATION','SRS_COMMUNICATION_T'], axis=1)

In [21]:
fs_obj, ml_obj

({'lsvm': RFECV(cv=5, estimator=LinearSVC(max_iter=1000000000), n_jobs=1,
        scoring='balanced_accuracy', verbose=3),
  'lr': RFECV(cv=5, estimator=LogisticRegression(max_iter=1000000000), n_jobs=1,
        scoring='balanced_accuracy', verbose=3)},
 defaultdict(dict,
             {'lsvm': {'svm': RandomizedSearchCV(cv=5, estimator=SVC(max_iter=1000000000), n_iter=250,
                                  n_jobs=3,
                                  param_distributions={'C': [0.1, 1, 5, 10],
                                                       'coef0': [0.0, 0.01, 0.5, 5, 50, 100],
                                                       'degree': [2, 3, 4, 5, 6],
                                                       'gamma': ['scale', 'auto'],
                                                       'kernel': ['poly', 'rbf', 'sigmoid']},
                                  scoring='balanced_accuracy', verbose=3),
               'nn': RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_i

In [54]:
# y = dfs.pop('my_labels')
results_dict = defaultdict(dict)
best_estimators_dict = defaultdict(dict)
for krf, rfe_model in fs_obj.items():
    Xselected = dfs[dfs.columns[rfe_model.support_]]
    for kml, ml_model in ml_obj[krf].items():
        bind = ml_obj[krf][kml].best_index_
        split_keys = [k for k in ml_obj[krf][kml].cv_results_.keys() if 'split' in k and 'test' in k]
        results_num = []
        best_estimators_dict[krf][kml] = ml_obj[krf][kml].best_estimator_
        for sk in split_keys:
            results_num.append(ml_obj[krf][kml].cv_results_[sk][bind])
        results_dict[krf][kml] =f"{np.mean(results_num):.2f} +/- {np.std(results_num, ddof=1):.2f}"

In [49]:
results_dict

defaultdict(dict,
            {'lsvm': {'svm': '0.87 +/- 0.06 ',
              'nn': '0.88 +/- 0.05 ',
              'lsvm': '0.87 +/- 0.05 '},
             'lr': {'svm': '0.87 +/- 0.06 ',
              'nn': '0.88 +/- 0.05 ',
              'lsvm': '0.87 +/- 0.05 '}})

In [50]:
df = pd.DataFrame(results_dict).T

In [51]:
df.loc['lsvm', 'svm']

'0.87 +/- 0.06 '

In [53]:
xselect_dir = "D:\\PhD\\codes\\behavioralProject\\models\\20210515_215356\\Xselected.p"

In [55]:
with open(xselect_dir, 'rb') as f:
    Xs, y = dill.load(f)

In [58]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [75]:
results_dict = defaultdict(dict)
for krf, rfe_model in best_estimators_dict.items():
    for kml, ml_model in best_estimators_dict[krf].items():
        Xstrain, Xstest, ytrain, ytest = train_test_split(Xs[krf], y, test_size=0.2, random_state=231, shuffle=True)
        best_estimators_dict[krf][kml].fit(Xstrain, ytrain)
        yhat = best_estimators_dict[krf][kml].predict(Xstest)
        C = confusion_matrix(ytest, yhat)
        results_dict[krf][kml] = {} 
        tp = C[0, 0]
        tn = C[1, 1]
        fp = C[1, 0]
        fn = C[0, 1]
        sensitivity = tp/(tp+fn) # sensitivity, recall, hit rate, or true positive rate
        specificity = tn/(tn+fp) # specificity, selectivity or true negative rate
        PPV = tp/(tp+fp) # precision or positive predictive value
        NPV = tn/(tn+fn)
        acc = (tp+tn)/(tp+tn+fp+tn)
        f1 = 2*tp/(2*tp+fp+fn)
        results_dict[krf][kml]['sens'] = sensitivity
        results_dict[krf][kml]['spec'] = specificity
        results_dict[krf][kml]['acc'] = acc
        results_dict[krf][kml]['f1'] = f1
        results_dict[krf][kml]['PPV'] = PPV
        results_dict[krf][kml]['NPV'] = NPV
        
#         print(f"rfe: {krf}, ml: {kml}, Confusion matrix: {C} \n")
#         print(f"rfe: {krf}, ml: {kml}, sensitivity: {sensitivity} \n")
#         print(f"rfe: {krf}, ml: {kml}, specificity: {specificity} \n")
#         print(f"rfe: {krf}, ml: {kml}, PPV: {PPV} \n")
#         print(f"rfe: {krf}, ml: {kml}, NPV: {NPV} \n")
#         print(f"rfe: {krf}, ml: {kml}, acc: {acc} \n")
#         print(f"rfe: {krf}, ml: {kml}, f1: {f1} \n")

In [68]:
dir(pd.MultiIndex)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__inv__',
 '__isub__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_accessors',
 '_add_comparison_methods',
 '_add_logical_methods',
 '_add_logical_methods_disabled',
 '_add_nu

In [89]:
index = pd.MultiIndex.from_product([['lr','lsvm'], ['sens','spec','acc','f1','PPV','NPV']], names = ['RFE','Metrics'])

In [90]:
df = pd.DataFrame(None, index=index, columns = best_estimators_dict['lr'].keys())

In [91]:
for krf in results_dict:
    for kml in results_dict[krf]:
        for metric in results_dict[krf][kml]:
            df.loc[(krf, metric), kml] = results_dict[krf][kml][metric]

In [92]:
df

svm        nn      lsvm
RFE  Metrics                              
lr   sens     0.820513  0.846154  0.820513
     spec     0.892857  0.892857  0.892857
     acc      0.670588  0.674419  0.670588
     f1       0.864865      0.88  0.864865
     PPV      0.914286  0.916667  0.914286
     NPV       0.78125  0.806452   0.78125
lsvm sens     0.769231  0.769231   0.74359
     spec     0.857143  0.785714  0.785714
     acc      0.658537      0.65   0.64557
     f1       0.821918       0.8  0.783784
     PPV      0.882353  0.833333  0.828571
     NPV      0.727273  0.709677    0.6875